<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/DNN_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py -O preprocess_sku.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2023-02-27 20:57:45--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  10.7MB/s    in 0.5s    

2023-02-27 20:57:45 (10.7 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2023-02-27 20:57:45--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5052 (4.9K) [text/plain]
Saving to: ‘preprocess_sku.py’

preprocess_sku.py   100%[===================>]

In [2]:
# ! pip install -q deepmatch
! pip install Keras-Preprocessing 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 1.1 MB/s eta 0:00:00


In [21]:
import pandas as pd
import numpy as np
import datetime
from ast import literal_eval
# from preprocess_sku import gen_data_set, gen_model_input, gen_data_set_timesplit
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from model_utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize,\
 reduce_mean, SampledSoftmaxLayer, get_item_embedding, sampledsoftmaxloss
from keras_preprocessing.sequence import pad_sequences


time: 12.2 ms (started: 2023-02-27 21:51:12 +00:00)


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 1.2 ms (started: 2023-02-27 20:58:12 +00:00)


In [8]:
import gc
gc.collect()

91

time: 411 ms (started: 2023-02-27 21:50:08 +00:00)


# Data

In [ ]:
data = pd.read_csv('sample_data/mik_sales_sp_10052022.csv', index_col = False, 
                   parse_dates = ['trans_date','created_time']
                   )
# 'sku_purchase_seq', 'category_path_purchase_seq', 'sku_view_sequence'
# data['hist_item_id'] = data['sku_purchase_seq'].apply(literal_eval)
# data['hist_category'] = data['most_recent_view_sku'].apply(literal_eval)
data.shape

(23206, 14)

time: 168 ms (started: 2023-02-24 15:41:38 +00:00)


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23206 entries, 0 to 23205
Data columns (total 14 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   data_source                         23206 non-null  object        
 1   user_id                             23206 non-null  int64         
 2   sku_number                          23206 non-null  object        
 3   qty                                 23206 non-null  int64         
 4   trans_date                          23206 non-null  datetime64[ns]
 5   created_time                        23206 non-null  datetime64[ns]
 6   email_address                       23206 non-null  object        
 7   geo_zip                             23086 non-null  object        
 8   most_frequent_device_class_general  23206 non-null  object        
 9   total_order_value                   17645 non-null  float64       
 10  total_coupon_value    

In [ ]:
data.shape, data['user_id'].nunique(), data['sku_number'].nunique()

((23206, 14), 200, 8036)

time: 6.06 ms (started: 2023-02-24 15:41:51 +00:00)


# New Data

In [15]:
# ! pip3 install --upgrade pandas==1.4.2
# ! pip list

time: 1.51 ms (started: 2023-02-27 20:57:17 +00:00)


In [9]:
data = pd.read_pickle('sample_data/df_input_20230227.sav')
data.shape

(70170, 12)

time: 1.12 s (started: 2023-02-27 21:50:12 +00:00)


# Feature Engineering

In [11]:
data.columns

Index(['user_id', 'sku_number', 'category_path', 'trans_date', 'created_time',
       'sku_purchase_seq', 'category_path_purchase_seq', 'sku_view_sequence',
       'seq_len', 'label', 'sku_number_org', 'user_id_org'],
      dtype='object')

time: 13.2 ms (started: 2023-02-27 21:50:24 +00:00)


In [12]:
sparse_features = ['sku_number', 'category_path',
                   'user_id']

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


time: 5.98 ms (started: 2023-02-27 21:50:26 +00:00)


In [13]:
feature_max_idx = {}
for feature in sparse_features:
    if feature == 'sku_number':
        feature_max_idx[feature] = max(data[feature].max(), data['sku_view_sequence'].explode().max()) + 1
    else:
        feature_max_idx[feature] = data[feature].max() + 1

time: 397 ms (started: 2023-02-27 21:50:28 +00:00)


In [14]:
user_profile = data[['user_id', 'user_id_org']].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["sku_number", 'sku_number_org']].drop_duplicates('sku_number')
user_profile.shape, item_profile.shape

((177, 1), (7729, 2))

time: 40.5 ms (started: 2023-02-27 21:50:30 +00:00)


# Train Test Split

In [16]:
test_start_date = data['trans_date'].max() - datetime.timedelta(7)
test_start_date

Timestamp('2023-02-20 00:00:00')

time: 6.89 ms (started: 2023-02-27 21:50:36 +00:00)


In [17]:
feature_cols = ['user_id', 'sku_number', 'category_path',
                'sku_purchase_seq', 'category_path_purchase_seq', 'sku_view_sequence',
                'seq_len']
train_data, train_label = data[data['trans_date'] < test_start_date][feature_cols], data[data['trans_date'] < test_start_date][['label']]
test_data, test_label = data[data['trans_date'] >= test_start_date][feature_cols], data[data['trans_date'] >= test_start_date][['label']]
train_data.shape, train_label.shape, test_data.shape, test_label.shape

((53426, 7), (53426, 1), (16744, 7), (16744, 1))

time: 70.1 ms (started: 2023-02-27 21:50:40 +00:00)


In [18]:
train_model_input = train_data.to_dict("list")
test_model_input = test_data.to_dict("list")

train_label = train_label.to_numpy().ravel()
test_label = test_label.to_numpy().ravel()

time: 25.3 ms (started: 2023-02-27 21:50:44 +00:00)


In [22]:
for feature in ['sku_number', 'category_path','user_id']:
  train_model_input[feature] = np.array(train_model_input[feature])
  test_model_input[feature] = np.array(test_model_input[feature])
for feature in ['sku_purchase_seq', 'category_path_purchase_seq', 'sku_view_sequence']:
  train_model_input[feature] = pad_sequences(train_model_input[feature], maxlen=SEQ_LEN, padding='post', truncating='post', value=0)
  test_model_input[feature] = pad_sequences(test_model_input[feature], maxlen=SEQ_LEN, padding='post', truncating='post', value=0)

time: 576 ms (started: 2023-02-27 21:51:30 +00:00)


# Create Embeddings

In [27]:
from collections import Counter
train_counter = Counter(train_model_input['sku_number'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['sku_number'])]
len(train_counter), type(train_counter), len(item_count)

(7693, collections.Counter, 10695)

time: 16.2 ms (started: 2023-02-27 21:52:47 +00:00)


In [28]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

time: 2.21 ms (started: 2023-02-27 21:52:49 +00:00)


### Create Feature Embeddings



In [29]:
# item features
item_feature_columns = ['sku_number']
# User features
user_feature_columns = ['user_id',
                        'category_path',
                        'sku_purchase_seq',
                        'category_path_purchase_seq',
                        'sku_view_sequence',
                        'seq_len']

time: 2.12 ms (started: 2023-02-27 21:52:53 +00:00)


In [35]:
user_sparse_feature_columns = ['user_id','category_path',]
user_seq_sparse_feature_columns = [
    'sku_purchase_seq',
    'category_path_purchase_seq',
    'sku_view_sequence'
]

time: 2.38 ms (started: 2023-02-27 21:54:10 +00:00)


In [36]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

time: 1.97 ms (started: 2023-02-27 21:54:24 +00:00)


In [37]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns:
  if name[0:3] == 'sku':
    emb = Embedding(feature_max_idx['sku_number'], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=False) # True
    emb.trainable = True
    sparse_embedding[name] = emb
  else:
    emb = Embedding(feature_max_idx['category_path'], embedding_dim,
                        embeddings_initializer=embeddings_initializer,
                        embeddings_regularizer=l2(
                            l2_reg),
                        name='sparse_' + 'seq_emb_'  + name,
                        mask_zero=False) # True
    emb.trainable = True
    sparse_embedding[name] = emb

time: 31.8 ms (started: 2023-02-27 21:54:25 +00:00)


In [38]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

time: 2.7 ms (started: 2023-02-27 21:54:26 +00:00)


In [39]:
sparse_embedding # include both user and item embedding layers

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7faea35d7c70>,
 'category_path': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7faea3738c70>,
 'sku_purchase_seq': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7faea3738a00>,
 'category_path_purchase_seq': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7faea2f54c10>,
 'sku_view_sequence': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7faea2f54190>,
 'sku_number': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7faea35bae50>}

time: 7.59 ms (started: 2023-02-27 21:54:28 +00:00)


# Create User Inputs


## Create Inputs Tensor

In [40]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

time: 48.6 ms (started: 2023-02-27 21:54:34 +00:00)


In [41]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'category_path:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'sku_purchase_seq:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'category_path_purchase_seq:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'sku_view_sequence:0' shape=(None, 50) dtype=int32>]

time: 3.58 ms (started: 2023-02-27 21:54:35 +00:00)


In [42]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


time: 4.14 ms (started: 2023-02-27 21:54:37 +00:00)


In [43]:
item_inputs_list

[<tf.Tensor 'sku_number:0' shape=(None, 1) dtype=int32>]

time: 3.75 ms (started: 2023-02-27 21:54:37 +00:00)


## Embed Inputs

In [44]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

time: 43.6 ms (started: 2023-02-27 21:54:40 +00:00)


In [45]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

time: 57.4 ms (started: 2023-02-27 21:54:41 +00:00)


In [48]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

[<tf.Tensor 'sparse_emb_user_id/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_category_path/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'average_pooling1d/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_1/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_2/Squeeze:0' shape=(None, 1, 32) dtype=float32>]

time: 6.3 ms (started: 2023-02-27 21:54:43 +00:00)


## Combine Embeded Input (User only)

In [49]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

time: 2.41 ms (started: 2023-02-27 21:54:44 +00:00)


In [50]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

<tf.Tensor 'flatten/Reshape:0' shape=(None, 128) dtype=float32>

time: 10.3 ms (started: 2023-02-27 21:54:45 +00:00)


# Sepecify Model Layers

### User Layers

In [51]:
from tensorflow.keras import layers

time: 1.43 ms (started: 2023-02-27 21:54:47 +00:00)


In [52]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

time: 1.86 ms (started: 2023-02-27 21:54:47 +00:00)


In [53]:
# from deepctr.layers import DNN
from model_utils import DNN

time: 1.36 ms (started: 2023-02-27 21:54:48 +00:00)


In [54]:
dnn_layers = DNN(hidden_units=user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



time: 2.39 ms (started: 2023-02-27 21:54:48 +00:00)


In [55]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

time: 438 ms (started: 2023-02-27 21:54:50 +00:00)


### Item Layer

In [56]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

# embed the item index input
item_embeding = sparse_embedding[item_feature_name]
item_embedding_weight = NoMask()(item_embeding(item_index))
pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
pooling_item_embedding_weight


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


<tf.Tensor 'lambda_1/l2_normalize:0' shape=(10695, 32) dtype=float32>

time: 313 ms (started: 2023-02-27 21:54:51 +00:00)


In [57]:
pooling_item_embedding_weight.eval(session=K.get_session()).shape

(10695, 32)

time: 415 ms (started: 2023-02-27 21:54:55 +00:00)


### Output Layer

In [59]:
output = SampledSoftmaxLayer()(
    [pooling_item_embedding_weight, user_dnn_output, item_features[item_feature_name]])
output

<tf.Tensor 'sampled_softmax_layer/ExpandDims:0' shape=(None, 1) dtype=float32>

time: 281 ms (started: 2023-02-27 21:54:57 +00:00)


In [62]:
pooling_item_embedding_weight

<tf.Tensor 'lambda_1/l2_normalize:0' shape=(10695, 32) dtype=float32>

time: 4.52 ms (started: 2023-02-27 21:54:59 +00:00)


In [63]:
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

time: 4.33 ms (started: 2023-02-27 21:54:59 +00:00)


In [64]:
item_features[item_feature_name]

<tf.Tensor 'sku_number:0' shape=(None, 1) dtype=int32>

time: 4.34 ms (started: 2023-02-27 21:55:00 +00:00)


## Build Model

In [65]:
from tensorflow.python.keras.models import Model

time: 1.43 ms (started: 2023-02-27 21:55:01 +00:00)


In [66]:
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_output)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))


time: 41.2 ms (started: 2023-02-27 21:55:02 +00:00)


# Model Compile and Training

In [67]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

time: 225 ms (started: 2023-02-27 21:55:05 +00:00)


In [68]:
history = model.fit(train_model_input, train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

Train on 53426 samples
Epoch 1/20
53426/53426 [==============================] - 8s 142us/sample - loss: 4.8973
Epoch 2/20
53426/53426 [==============================] - 5s 89us/sample - loss: 4.5147
Epoch 3/20
53426/53426 [==============================] - 5s 101us/sample - loss: 4.3372
Epoch 4/20
53426/53426 [==============================] - 6s 116us/sample - loss: 4.2263
Epoch 5/20
53426/53426 [==============================] - 5s 89us/sample - loss: 4.1844
Epoch 6/20
53426/53426 [==============================] - 7s 126us/sample - loss: 4.1698
Epoch 7/20
53426/53426 [==============================] - 5s 90us/sample - loss: 4.1535
Epoch 8/20
53426/53426 [==============================] - 5s 91us/sample - loss: 4.1313
Epoch 9/20
53426/53426 [==============================] - 7s 129us/sample - loss: 4.1167
Epoch 10/20
53426/53426 [==============================] - 5s 93us/sample - loss: 4.1213
Epoch 11/20
53426/53426 [==============================] - 7s 131us/sample - loss: 4.1070
E

In [69]:
train_model_input["sku_purchase_seq"].shape

(53426, 50)

time: 3.78 ms (started: 2023-02-27 21:57:38 +00:00)


# Predict: Generate Item and User Embeddings

In [70]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"sku_number": item_profile['sku_number'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

/usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:2455: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


(16744, 32)
(7729, 32)
time: 371 ms (started: 2023-02-27 21:57:46 +00:00)


# Recommendation

In [71]:
! pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 40.6 MB/s eta 0:00:00
time: 5.48 s (started: 2023-02-27 21:57:53 +00:00)


In [72]:
import numpy as np
import faiss
from tqdm import tqdm
# from deepmatch.utils import recall_N

time: 43.1 ms (started: 2023-02-27 21:57:59 +00:00)


In [73]:
def recall_N(y_true, y_pred, N=30):
    return len(set(y_pred[:N]) & set(y_true)) * 1.0 / len(y_true)

time: 2.56 ms (started: 2023-02-27 21:57:59 +00:00)


In [74]:
def precision_N(y_true, y_pred, N=30):
    return len(set(y_pred[:N]) & set(y_true)) * 1.0 / N

time: 2.36 ms (started: 2023-02-27 21:57:59 +00:00)


In [87]:
test_true_label = {uid:label for uid, label in zip(test_model_input['user_id'], 
                                                   test_model_input['sku_number'])}
import numpy as np
import faiss
from tqdm import tqdm

index = faiss.IndexFlatIP(embedding_dim)
index.add(item_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 50)
recall = []
precision = []
f1 = []
hit = 0
pred_label = {}
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['sku_number'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=30)
        precision_score = precision_N(test_true_label[uid], pred, N=30)

        recall.append(recall_score)
        precision.append(precision_score)
        f1_score = 2 * (precision_score * recall_score) /(precision_score + recall_score)
        f1.append(f1_score)
        pred_label[uid] = pred
        if test_true_label[uid] in pred:
            hit += 1
    except:
        pass
print("")
print("recall", np.mean(recall))
print("precision", np.mean(precision))
print("f1", np.mean(f1))
print("hit rate", hit / len(test_user_model_input['user_id']))

16744it [00:05, 2867.04it/s]


recall nan
precision nan
f1 nan
hit rate 0.0
time: 7.5 s (started: 2023-02-27 22:06:28 +00:00)



/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
